In [1]:
import os
import shutil

In [2]:
%store -r 
#restore train, validate, and test dataframes from other notebook
train.head()

,file_name,hair_color,eyeglasses,smiling,young,human
2308,2531,2,1,1,1,-1
82,92,1,1,1,1,-1
4401,4830,-1,-1,-1,1,1
817,907,0,-1,-1,1,1
471,522,1,-1,1,1,1


In [3]:
def reset_images_to_main_dir(delete_dirs=True):
    dest_dir = './images/'
    walker = os.walk(dest_dir)
    rem_dirs = walker.__next__()[1]
 
    for data in walker:
        for files in data[2]:
            try:
                shutil.move(data[0] + os.sep + files, dest_dir)
            except shutil.Error:
                print(shutil.Error)
                continue
    if delete_dirs:
        for dirs in rem_dirs:
            shutil.rmtree(dest_dir + os.sep + dirs)

def symlink_classes_images(train, validate, test, 
                           imagesPath,
                           trainPath,
                           validatePath,
                           testPath,
                           rowName,
                           reset_images=False, 
                           delete_dirs=False):
    if reset_images:
        reset_images_to_main_dir(delete_dirs=delete_dirs)
    newPaths = [trainPath, validatePath, testPath]
    dfList = [train, validate, test]
    for path in newPaths:
        if os.path.isdir(path)==False:
            os.mkdir(path)
    
    for df in dfList:
        for idx, row in df.iterrows():
            className = int(row[rowName])
            pathAppend = str(className)+'/'
            
            fileName = str(int(row['file_name']))+'.png'
            filePath = os.path.join(imagesPath, fileName)
            trainClassPath = os.path.join(trainPath, pathAppend)
            trainFilePath = os.path.join(trainClassPath, fileName)
            validateClassPath = os.path.join(validatePath, pathAppend)
            validateFilePath = os.path.join(validateClassPath, fileName)
            testClassPath = os.path.join(testPath, pathAppend)
            testFilePath = os.path.join(testClassPath, fileName)
            classPaths = [trainClassPath, validateClassPath, testClassPath]
            for path in classPaths:
                if os.path.isdir(path)==False:
                    os.mkdir(path)
            if(df.equals(test)):
                newFilePath = testFilePath
            if(df.equals(train)):
                newFilePath = trainFilePath
            if(df.equals(validate)):
                newFilePath = validateFilePath
            if os.path.isfile(filePath):
                os.symlink(filePath, newFilePath)
            elif os.path.isfile(testFilePath):
                os.symlink(testFilePath, newFilePath)
            elif os.path.isfile(validateFilePath):
                os.symlink(validateFilePath, newFilePath)
            elif os.path.isfile(trainFilePath):
                os.symlink(trainFilePath, newFilePath)
            else:
                print("File missing: ", fileName)

imagesPath = os.path.abspath('./images/')
trainPath = os.path.abspath('./images/train/young/')
validatePath = os.path.abspath('./images/validate/young/')
testPath = os.path.abspath('./images/test/young/')
rowName = 'young'
symlink_classes_images(train, validate, test, 
                       imagesPath, trainPath, validatePath, testPath,
                       rowName, reset_images=False, delete_dirs=False)

FileExistsError: [Errno 17] File exists: '/home/ilyas/dev/AMLSassignment/images/2531.png' -> '/home/ilyas/dev/AMLSassignment/images/train/young/1/2531.png'

In [5]:
train_young = train[['file_name', 'young']]
train_young = train_young.replace(to_replace=-1, value=0).reset_index(drop=True)
validate_young = validate[['file_name', 'young']]
validate_young = validate_young.replace(to_replace=-1, value=0).reset_index(drop=True)
test_young = test[['file_name', 'young']]
test_young = test_young.replace(to_replace=-1, value=0).reset_index(drop=True)

In [6]:
n_young = train_young['young'].value_counts()
print(n_young)

1    2176
0     563
Name: young, dtype: int64


In [7]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.layers.advanced_activations import LeakyReLU

K.set_image_dim_ordering('th')

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(3, 128, 128)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
#model.add(LeakyReLU(alpha=0.3))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Using TensorFlow backend.


In [8]:
import os
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

def get_ImageDataGenerator(shear_range=0, 
                           zoom_range=0, 
                           horizontal_flip=False):
    datagen = ImageDataGenerator(
                rescale=1./255,
                shear_range=0,
                zoom_range=0,
                horizontal_flip=False,
                data_format='channels_first')
    
    return datagen

def get_flow_from_directory(ImageDataGenerator,
                            img_dir,
                            target_size=(128,128),
                            class_mode='binary',
                            batch_size=32,
                            shuffle=True,
                            seed=123):
    gen = ImageDataGenerator.flow_from_directory(
                            directory=img_dir,
                            target_size=target_size,
                            class_mode=class_mode,
                            batch_size=batch_size,
                            shuffle=shuffle,
                            seed=seed,
                            follow_links=True)
    return gen


# this is the augmentation configuration we will use for training
train_datagen = get_ImageDataGenerator(shear_range=0,
                                       zoom_range=0,
                                       horizontal_flip=False)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = get_ImageDataGenerator()
#Takes the dataframe and the path to a directory and 
#generates batches of augmented/normalized data.

trainPath = './images/train/young/'
validatePath = './images/validate/young/'
batch_size = 32
target_size=(128,128)
class_mode='binary'
shuffle=True
seed=123

train_gen = get_flow_from_directory(train_datagen,
                                    trainPath,
                                    target_size,
                                    class_mode,
                                    batch_size,
                                    shuffle,
                                    seed)

validation_gen = get_flow_from_directory(test_datagen,
                                    validatePath,
                                    target_size,
                                    class_mode,
                                    batch_size,
                                    shuffle,
                                    seed)

Found 2739 images belonging to 2 classes.
Found 913 images belonging to 2 classes.


In [9]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from sklearn.utils import class_weight

#class_weights = class_weight.compute_class_weight('balanced',
                                                 #['cartoon', 'human'],
                                                 #train_human.head(2720)['human'])

earlyStopping = EarlyStopping(monitor='val_loss',
                              min_delta=0.001,
                              patience=0,
                              verbose=1, 
                              mode='auto',
                              restore_best_weights=True)

LR = ReduceLROnPlateau(monitor='val_loss', 
                       factor=0.2, 
                       patience=0, 
                       verbose=1, 
                       mode='auto', 
                       min_delta=0.01, 
                       cooldown=0, 
                       min_lr=0.001)
train_gen.reset()
validation_gen.reset()

train_steps=train_gen.n/train_gen.batch_size
valid_steps=validation_gen.n/validation_gen.batch_size

In [10]:
print(train_steps, valid_steps)
print(train_steps*train_gen.batch_size, valid_steps*validation_gen.batch_size)
model.summary()

85.59375 28.53125
2739.0 913.0
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 128, 128)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 126, 126)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 63, 63)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 63, 63)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 63, 63)        9248      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 64, 61, 61)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 64, 30, 3

In [ ]:
model.fit_generator(generator=train_gen,
                    steps_per_epoch=train_steps,
                    validation_data=validation_gen,
                    validation_steps=valid_steps,
                    epochs=50,
                    callbacks=[earlyStopping, LR],
                    class_weight = 'auto')

#model.save_weights('.h5')
model.save_weights('young_detection_from_dir.h5')

Epoch 1/50
86/85 [==============================] - 18127s 211s/step - loss: 0.4771 - acc: 0.8142 - val_loss: 0.3598 - val_acc: 0.8697
Epoch 2/50
19/85 [=====>........................] - ETA: 5:13 - loss: 0.3416 - acc: 0.8635

In [ ]:
validation_gen = get_flow_from_directory(test_datagen,
                                        validatePath,
                                        target_size,
                                        class_mode,
                                        1,
                                        False)

validation_gen.reset()

model.evaluate_generator(generator=validation_gen, steps=913, verbose=1)

In [ ]:
testPath = './images/test/young/'


test_gen = get_flow_from_directory(test_datagen,
                                    testPath,
                                    target_size,
                                    class_mode,
                                    1,
                                    False)

test_gen.reset()

model.evaluate_generator(generator=test_gen, steps=913, verbose=1)

In [ ]:
test_gen.reset()
probabilities = model.predict_generator(test_gen,verbose=1,steps=len(test_gen))

In [ ]:
import pandas as pd
import numpy as np
output_df = pd.DataFrame(
    {'filename': test_gen.filenames,
     'y_true': test_gen.classes,
     'y_pred': list(map(int, np.rint(probabilities).flatten()))
    })

In [ ]:
output_df_wrong = output_df.query('y_true != y_pred')
output_df_wrong

In [ ]:
from sklearn.metrics import confusion_matrix
y_true = test_gen.classes
y_pred = np.rint(probabilities)
confusion_matrix(y_true=y_true, y_pred=y_pred)